In [1]:
pip install networkx

   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 1.7/1.7 MB 15.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd
import requests
import folium
import networkx as nx
import random
import math

df = pd.read_csv('D:/Code/Coding Stuff/2nd Year/AI Concpet/open_pubs.csv')

#Cleaning
df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')
df = df.dropna(subset=['latitude', 'longitude'])
df = df.drop_duplicates(subset=['name'], keep='last')

chosen_city = "City of London"
df_chosen_city = df[df['local_authority'] == chosen_city].head(10)  

def get_osrm_route(start_coords, end_coords):
    url = f"http://router.project-osrm.org/route/v1/driving/{start_coords[1]},{start_coords[0]};{end_coords[1]},{end_coords[0]}?overview=full&geometries=geojson"
    response = requests.get(url)
    
    if response.status_code == 200:
        route_data = response.json()
        distance_meters = route_data['routes'][0]['distance']  
        route_coords = route_data['routes'][0]['geometry']['coordinates']
        return route_coords, distance_meters
    else:
        return None, None

m = folium.Map(location=[df_chosen_city['latitude'].mean(), df_chosen_city['longitude'].mean()], zoom_start=13)

pub_locations = list(zip(df_chosen_city['latitude'], df_chosen_city['longitude'])) 
pub_names = df_chosen_city['name'].tolist()  


for index, row in df_chosen_city.iterrows():
    folium.Marker(location=[row['latitude'], row['longitude']],
                  popup=f"{row['name']}<br>{row['address']}<br>{row['postcode']}",).add_to(m)

G = nx.Graph()

for i, pub_name in enumerate(pub_names):
    G.add_node(pub_name, pos=pub_locations[i])

total_distance = 0

for i in range(len(pub_locations) - 1):
    for j in range(i + 1, len(pub_locations)): 
        start_coords = pub_locations[i]
        end_coords = pub_locations[j]
        
        route, distance_meters = get_osrm_route(start_coords, end_coords)
        
        if route:
            distance_km = distance_meters / 1000
            total_distance += distance_km
            
            route_latlong = [(coord[1], coord[0]) for coord in route]
  
            folium.PolyLine(locations=route_latlong, color='blue', weight=2.5, opacity=0.8).add_to(m)
            
            G.add_edge(pub_names[i], pub_names[j], weight=distance_km)
            
m


In [ ]:
# Count the number of pubs (rows) in the chosen city


Total number of pubs in Liverpool: 370


In [4]:
import time 

In [10]:

def calculate_route_distance(route, graph):
    total_distance = 0
    for i in range(len(route) - 1):
        total_distance += graph[route[i]][route[i + 1]]['weight']
    total_distance += graph[route[-1]][route[0]]['weight']
    return total_distance

def simulated_annealing(graph, initial_route):
    current_route = initial_route.copy()
    current_distance = calculate_route_distance(current_route, graph)
    best_route = current_route
    best_distance = current_distance

    temperature = 1000.0
    cooling_rate = 0.9999
    while temperature > 1:
        new_route = current_route.copy()
        i, j = random.sample(range(len(new_route)), 2)
        new_route[i], new_route[j] = new_route[j], new_route[i]
        
        new_distance = calculate_route_distance(new_route, graph)
        if new_distance < current_distance or random.random() < math.exp((current_distance - new_distance) / temperature):
            current_route = new_route
            current_distance = new_distance
            
            if current_distance < best_distance:
                best_route = current_route
                best_distance = current_distance

        temperature *= cooling_rate

    return best_route, best_distance

start_time = time.time()


initial_route = pub_names

optimized_route, optimized_distance = simulated_annealing(G, initial_route)

optimized_route.append(optimized_route[0]) 

end_time = time.time()
elapsed_time = end_time - start_time

print("Optimized Route:", " -> ".join(optimized_route))
print(f"Optimized Distance: {optimized_distance:.2f} km")
print(f"Simulated Annealing Time: {elapsed_time:.4f} seconds")

m_optimized = folium.Map(location=[df_chosen_city['latitude'].mean(), df_chosen_city['longitude'].mean()], zoom_start=13)

for index, row in df_chosen_city.iterrows():
    folium.Marker(location=[row['latitude'], row['longitude']],
                  popup=f"{row['name']}<br>{row['address']}<br>{row['postcode']}").add_to(m_optimized)

for i in range(len(optimized_route) - 1):
    start_pub = optimized_route[i]
    end_pub = optimized_route[i + 1]
    
    start_coords = G.nodes[start_pub]['pos']
    end_coords = G.nodes[end_pub]['pos']
    
    route, _ = get_osrm_route(start_coords, end_coords)
    if route:
        route_latlong = [(coord[1], coord[0]) for coord in route]
        
        folium.PolyLine(locations=route_latlong, color='red', weight=2.5, opacity=0.8).add_to(m_optimized)

# Display the optimized map
m_optimized
m_optimized.save('optimized_route_map_liverpool_150_nodes.html')


Optimized Route: Balls Brothers Shoe Lane -> Balls Brothers Ltd -> Babble City -> Balls Brothers Austin Friars -> Balls Brothers @ Minster Court -> Agenda -> Abbey -> Balls Brothers Wine Bar -> Astronomer -> Amber Bar -> Balls Brothers Shoe Lane
Optimized Distance: 11.98 km
Simulated Annealing Time: 0.4887 seconds


In [ ]:
class hashabledict(dict):
    def __hash__(self):
        return hash(tuple(sorted(self.items())))

def bfs(graph, start):
    queue = []

    queue.append((start, {key: False for (key) in G.nodes()} , 0, [start]))
    visited = {}

    while queue:
        current_city, visited_cities, current_cost, path = queue.pop(0)

        if all(val == True for val in visited_cities.values()):
            return path

        if (hashabledict(visited_cities) in visited and visited[visited_cities] <= current_cost):
            continue

        visited[hashabledict(visited_cities)] = current_cost        

        for next_city in list(graph[current_city]):
            if visited_cities[next_city] == False:
                visited_cities[next_city] = True
                current_cost += graph[current_city][next_city].get('weight', None)
                #print(current_cost)
                path.append(next_city)
                queue.append((next_city, visited_cities, current_cost, path))


# Start timer
start_time = time.time_ns()

min_cost = float('inf') # Store minimal const

# Loop through the all starting points to get the best length
for cities in list(G.nodes()):
    bfs_path = bfs(G, cities)
    bfs_cost = calculate_route_distance(bfs_path, G)

    if bfs_cost < min_cost:
        min_cost = bfs_cost
        min_path = bfs_path        

end_time = time.time_ns() - start_time

print("BFS Cost : {:.2f} km".format(calculate_route_distance(bfs_path, G)))
print("BFS Path :")
print(" -> ".join(min_path))
print("BFS Time :", end_time/1000000000, "seconds")

# Represents the path in map
m_bfs = folium.Map(location=[df_chosen_city['latitude'].mean(), df_chosen_city['longitude'].mean()], zoom_start=13)

# Add pub markers to the optimized map
for index, row in df_chosen_city.iterrows():
    folium.Marker(location=[row['latitude'], row['longitude']],
                  popup=f"{row['name']}<br>{row['address']}<br>{row['postcode']}").add_to(m_bfs)
    
for i in range(len(bfs_path) - 1):
    start_pub = bfs_path[i]
    end_pub = bfs_path[i + 1]
    
    # Get the coordinates for the start and end pubs
    start_coords = G.nodes[start_pub]['pos']
    end_coords = G.nodes[end_pub]['pos']
    
    # Get route coordinates and distance from OSRM
    route, _ = get_osrm_route(start_coords, end_coords)
    if route:
        # Convert route coordinates to lat-lng for folium display
        route_latlong = [(coord[1], coord[0]) for coord in route]
        
        # Add route as polyline to the optimized map
        folium.PolyLine(locations=route_latlong, color='red', weight=2.5, opacity=0.8).add_to(m_bfs)

# Display the optimized map
m_bfs.save('bfs_optimized_route_map_liverpool_40_nodes.html')
m_bfs

BFS Cost : 192.88 km
BFS Path :
Brewdog -> Abbey Road Wine Bar -> Afro Caribbean & Friends Lunch Club -> Aigburth Cricket Club -> Aintree And Fazakerley Working Mens -> Albany Hotel -> All Saints Parochial Centre And Club -> Angels Paradise -> Angels Vip -> Ashdale Inn -> Atlantic -> Baltic Bakehouse -> Bar Ca Va -> Bar Coast -> Beaconsfield Community Centre -> Bear And Staff -> Beauty Bazaar Harvey Nichols -> Beehive -> Belmont Hotel -> Berry & Rye -> Bier -> Black Lodge Brewing Co Ltd -> Blackburne Arms -> Blarney Stone -> Bleak House -> Blue Angel -> Brambles -> Bramley Moore -> Breeze Hotel -> British Railways Sports And Social C -> Broadgreen Conservative Club -> Brook House -> Brownlow's -> C A D W A Social Club -> C I Edwardian Association -> Cabin Club -> Caledonia -> Canon Public House -> Carisbrooke Hotel -> Carnarvon Castle -> Brewdog
BFS Time : 0.0200326 seconds


In [129]:
def create_initial_population(population_size, pub_names):
    population = []
    for _ in range(population_size):
        route = pub_names[:]
        random.shuffle(route)
        population.append(route)
    return population

# Select the best individuals for mating (tournament selection)
def selection(population, graph, tournament_size=5):
    selected = []
    for _ in range(tournament_size):
        individual = random.choice(population)
        selected.append(individual)
    selected.sort(key=lambda x: calculate_route_distance(x, graph))
    return selected[0]  # Return the best individual in the tournament

# Crossover function to create a child route
def crossover(parent1, parent2):
    size = len(parent1)
    start, end = sorted(random.sample(range(size), 2))
    child = [None] * size
    # Inherit a subset from parent1
    child[start:end] = parent1[start:end]
    # Fill remaining positions with genes from parent2 in order
    ptr = 0
    for gene in parent2:
        if gene not in child:
            while child[ptr] is not None:
                ptr += 1
            child[ptr] = gene
    return child

# Mutation function to introduce some variations
def mutate(route, mutation_rate=0.01):
    for i in range(len(route)):
        if random.random() < mutation_rate:
            j = random.randint(0, len(route) - 1)
            route[i], route[j] = route[j], route[i]  # Swap two pubs
    return route

# Main Genetic Algorithm function
def genetic_algorithm(graph, pub_names, population_size=100, generations=500, mutation_rate=0.01):
    population = create_initial_population(population_size, pub_names)
    best_route = min(population, key=lambda x: calculate_route_distance(x, graph))
    best_distance = calculate_route_distance(best_route, graph)

    for generation in range(generations):
        new_population = []

        # Create new population through selection, crossover, and mutation
        for _ in range(population_size):
            parent1 = selection(population, graph)
            parent2 = selection(population, graph)
            child = crossover(parent1, parent2)
            child = mutate(child, mutation_rate)
            new_population.append(child)

        # Update population and track the best route
        population = new_population
        current_best_route = min(population, key=lambda x: calculate_route_distance(x, graph))
        current_best_distance = calculate_route_distance(current_best_route, graph)

        if current_best_distance < best_distance:
            best_route = current_best_route
            best_distance = current_best_distance

        # Optionally, print progress every 50 generations
        # if generation % 50 == 0:
        #     print(f"Generation {generation}: Best Distance = {best_distance:.2f} km")

    return best_route, best_distance

# Running the Genetic Algorithm
start_time = time.time()

# Initial route setup and running the algorithm
optimized_route, optimized_distance = genetic_algorithm(G, pub_names, population_size=100, generations=500, mutation_rate=0.01)

end_time = time.time()
elapsed_time = end_time - start_time


print("Optimized Route:", " -> ".join(optimized_route))
print(f"Optimized Distance: {optimized_distance:.2f} km")
print(f"Genetic Algorithm Time: {elapsed_time:.4f} seconds")

# Save results to a text file
with open("genetic_algorithm_results.txt", "a") as file:
    file.write("Optimized Route:\n" + " -> ".join(optimized_route) + "\n")
    file.write(f"Optimized Distance: {optimized_distance:.2f} km\n")
    file.write(f"Genetic Algorithm Time: {elapsed_time:.4f} seconds\n")

print("Results saved to genetic_algorithm_results.txt")

# Visualizing the optimized route on a map
m_optimized_genetic = folium.Map(location=[df_chosen_city['latitude'].mean(), df_chosen_city['longitude'].mean()], zoom_start=13)

# Add markers and optimized route to the map
for index, row in df_chosen_city.iterrows():
    folium.Marker(location=[row['latitude'], row['longitude']],
                  popup=f"{row['name']}<br>{row['address']}<br>{row['postcode']}").add_to(m_optimized_genetic)

# Plot optimized route on the map
optimized_route.append(optimized_route[0])  # Complete the cycle
for i in range(len(optimized_route) - 1):
    start_pub = optimized_route[i]
    end_pub = optimized_route[i + 1]
    
    start_coords = G.nodes[start_pub]['pos']
    end_coords = G.nodes[end_pub]['pos']
    
    route, _ = get_osrm_route(start_coords, end_coords)
    if route:
        route_latlong = [(coord[1], coord[0]) for coord in route]
        
        folium.PolyLine(locations=route_latlong, color='green', weight=2.5, opacity=0.8).add_to(m_optimized_genetic)

# Display the optimized map
m_optimized_genetic

Optimized Route: Abbey Road Wine Bar -> Beauty Bazaar Harvey Nichols -> Bar Ca Va -> Beaconsfield Community Centre -> Carisbrooke Hotel -> Aintree And Fazakerley Working Mens -> British Railways Sports And Social C -> Breeze Hotel -> Albany Hotel -> Atlantic -> Bar Coast -> Bramley Moore -> Angels Vip -> Angels Paradise -> Beehive -> Carnarvon Castle -> Brewdog -> Bier -> Blarney Stone -> Brownlow's -> Cabin Club -> Black Lodge Brewing Co Ltd -> Berry & Rye -> Blue Angel -> Baltic Bakehouse -> Belmont Hotel -> Canon Public House -> All Saints Parochial Centre And Club -> Afro Caribbean & Friends Lunch Club -> Ashdale Inn -> Brambles -> C I Edwardian Association -> Broadgreen Conservative Club -> C A D W A Social Club -> Bear And Staff -> Brook House -> Aigburth Cricket Club -> Bleak House -> Blackburne Arms -> Caledonia
Optimized Distance: 73.92 km
Genetic Algorithm Time: 8.8685 seconds
Results saved to genetic_algorithm_results.txt


KeyboardInterrupt: 